In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from typing import List
from langchain_chroma import Chroma
from langchain.schema import Document
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder,PromptTemplate
from langchain.chains import create_retrieval_chain,create_history_aware_retriever # create a chain which will retrieve from a vector database
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.output_parsers import StrOutputParser
import faiss
from langchain.vectorstores import FAISS

from typing import List
from pydantic import BaseModel, Field

import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer

e:\python\2.PROJECTS\cook_bot\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df=pd.read_csv("data\\BigBasket Products.csv")

In [4]:
drop:dict={'category':['Beauty & Hygiene','Kitchen, Garden & Pets','Cleaning & Household','Baby Care'],
                   'type':['Children (2-5 Yrs)','Kids (5+Yrs)','Syrups & Concentrates'],
                   'sub_category':['Baby Food & Formula','Ready To Cook & Eat'],
                   'brand':['Real Activ','Tropicana','Smoodies']}

for col,values in drop.items():
    df[~df[col].isin(values)]

mask = pd.Series(True, index=df.index)

# Iterate through the drop dictionary to exclude matching rows
for col, values in drop.items():
    if col in df.columns:
        mask &= ~df[col].isin(values) #Combines the current condition with the existing mask using a logical AND (&), ensuring only rows satisfying all conditions remain

# Apply the mask to filter the DataFrame
df = df[mask]


d=['Beauty & Hygiene','Kitchen, Garden & Pets','Cleaning & Household','Baby Care']
df = df[~df["category"].isin(d)]

t=['Children (2-5 Yrs)','Kids (5+Yrs)','Syrups & Concentrates']
df= df[~df['type'].isin(t)]

# sub-cat
sc=['Baby Food & Formula','Ready To Cook & Eat']
df= df[~df['sub_category'].isin(sc)]

# brand
b=['Real Activ','Tropicana','Smoodies']
df= df[~df['brand'].isin(b)]

drop:dict={'category':['Beauty & Hygiene','Kitchen, Garden & Pets','Cleaning & Household','Baby Care'],
                   'type':['Children (2-5 Yrs)','Kids (5+Yrs)','Syrups & Concentrates'],
                   'sub_category':['Baby Food & Formula','Ready To Cook & Eat'],
                   'brand':['Real Activ','Tropicana','Smoodies']}

df.drop_duplicates(subset=['product','category','sub_category','brand','type'],inplace=True)

df.reset_index(drop=True,inplace=True)
df.drop('index',axis=1,inplace=True)

filtered_df = df

In [34]:
import re
product = filtered_df['product']
product = product.astype(str)
product_list = product.to_list()
corpus = []
for index,value in enumerate(product_list):
    review = re.sub('[^a-zA-Z0-9]', ' ', product_list[index])
    review = review.lower()
    corpus.append(review)

In [35]:
len(corpus)

11344

In [36]:
import pickle
with open("artifacts/corpus.pkl", "wb") as file:
    pickle.dump(corpus, file)

# experiment

In [6]:
# from src.components.db_feeder import DbCreation

# db_create = DbCreation()
# vector_store,persist_directory=db_create.initiate_db_creation(filtered_df)

In [6]:
import os
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

#all-MiniLM-L6-v2
#sentence-transformers/all-MPNet-base-v2
#sentence-transformers/all-MPNet-base-v2

# fiass

In [13]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
# embeddings = model.encode(corpus)  # Embeds the entire corpus

In [4]:
# when u want to create we have already created
documents = [Document(page_content=item) for item in corpus]
db = FAISS.from_documents(documents,embeddings)

NameError: name 'corpus' is not defined

In [ ]:
db.similarity_search_with_relevance_scores('all-purpose flour')

[(Document(metadata={}, page_content='unbleached all purpose flour'),
  0.6803553506139839),
 (Document(metadata={}, page_content='wheat flour'), 0.6684023568623245),
 (Document(metadata={}, page_content='corn flour'), 0.6161863101408142),
 (Document(metadata={}, page_content='rice   flour'), 0.6144316103967601)]

In [ ]:
# db.save_local(folder_path="artifacts/",index_name="products")

In [7]:
# load 
db_load = FAISS.load_local(folder_path="artifacts",index_name="products",allow_dangerous_deserialization=True,embeddings=embeddings)

In [15]:
a= db_load.similarity_search_with_relevance_scores('tomato',k=5)
# document = a[0][0].
# score = a[0][1]
page_content = a[0][0].page_content
score = a[0][1]
print(page_content,score)
a

tomato   green 0.7037380428461913


[(Document(metadata={}, page_content='tomato   green'), 0.7037380428461913),
 (Document(metadata={}, page_content='tomato   cherry'), 0.6640314336438053),
 (Document(metadata={}, page_content='tomato   cherry'), 0.6640314336438053),
 (Document(metadata={}, page_content='puree   tomato'), 0.6304929314272916),
 (Document(metadata={}, page_content='tomato   local'), 0.6288342422041453)]

In [79]:
score

0.7490340195841223

In [7]:
retrievar = db_load.as_retriever()

In [8]:
retrievar

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001DBC1EEE140>, search_kwargs={})

In [9]:
# embeddings = np.array(embeddings).astype('float32')
# #  Normalize embeddings for cosine similarity
# embeddings /= np.linalg.norm(embeddings, axis=1, keepdims=True)
# dim = embeddings.shape[1]

In [10]:
# vector_store= faiss.IndexFlatL2(dim)

In [11]:
# vector_store.add(embeddings)

In [12]:
# faiss.write_index(vector_store, 'artifacts/vector_store.index')

## another method for fiass

In [6]:
import faiss
vector_store_read = faiss.read_index('artifacts/vector_store.index')

In [41]:
import numpy as np
import pandas as pd
def similarity_search(query, top_k):
    query_embedding = model.encode([query])
    query_embedding = np.array(query_embedding).astype('float32')
    query_embedding /= np.linalg.norm(query_embedding, axis=1, keepdims=True)
    dim = query_embedding.shape[1]

    result= vector_store_read.search(query_embedding, top_k)
    return result

In [ ]:
query = 'vanilla'
score,row = similarity_search(query,15)
print(score)
print(row)
filtered_df.loc[row[0]]

[[0.3810705  0.46213353 0.47988397 0.48674577 0.50859475 0.52341664
  0.53061163 0.55082136 0.5580959  0.5592432  0.5631994  0.56598353
  0.56598353 0.5791143  0.6068146 ]]
[[10178  9261  1232 11335 10120  8414  8112  7743  1722  5838  1525   611
  10624  3692  6925]]


,product,category,sub_category,brand,sale_price,market_price,type,rating,description
10178,Ice Cream - Vanilla,Gourmet & World Food,Dairy & Cheese,Haagen-Dazs,750.00,750.0,Gourmet Ice Cream,NaN,Vanilla ice cream prepared with new cream and ...
9261,Tea - Vanilla,Beverages,Tea,Pukka,450.00,450.0,Tea Bags,5.0,"Caffeine-free vanilla chai with sweet, warming..."
1232,Vanilla Powder,Gourmet & World Food,Cooking & Baking Needs,Puramate,31.50,35.0,"Baking, Cake Decorations",5.0,Vanilla Powder Deluxe is primarily used as a f...
11335,Vanilla Ice Cream,Gourmet & World Food,Dairy & Cheese,Haagen-Dazs,225.00,225.0,Gourmet Ice Cream,NaN,Some things are so good in their simplest form...
10120,Flavoured Milk - Vanilla,"Bakery, Cakes & Dairy",Dairy,Heritage,96.00,100.0,"Flavoured, Soya Milk",4.0,Heritage Vanilla Flavored Milk is wonderful to...
8414,Vanilla Dew,"Bakery, Cakes & Dairy","Cookies, Rusk & Khari",Milk Ma,30.00,30.0,"Bakery Biscuits, Cookies",3.9,Milkma Vanilla Dew are vanilla flavoured littl...
8112,Milkshake- Vanilla,"Bakery, Cakes & Dairy",Dairy,Cavins,172.80,210.0,"Flavoured, Soya Milk",4.3,The soothing aroma of vanilla with its sweet a...
7743,"Ice Cream - Vanilla, Premium",Gourmet & World Food,Dairy & Cheese,London Dairy,140.00,140.0,Gourmet Ice Cream,NaN,"Crafted with the finest, most fragrant vanilla..."
1722,Milk Shake - Vanilla,"Bakery, Cakes & Dairy",Dairy,Milky Mist,20.10,30.0,"Flavoured, Soya Milk",4.1,The thick and smooth taste of this creamy Milk...
5838,Vanilla Beans,Gourmet & World Food,Cooking & Baking Needs,Sprig,664.05,699.0,"Baking, Cake Decorations",4.4,"Vanilla, with its amazing flavour profile, can..."


In [384]:
query = 'milk'
score,row = similarity_search(query,3)
print(score)
print(row)

[[0.33338588 0.4388485  0.4480163 ]]
[[ 460 9128 5002]]


In [385]:
filtered_df.loc[row[0]]

,product,category,sub_category,brand,sale_price,market_price,type,rating,description
460,Chocolate - Milk,Gourmet & World Food,Cooking & Baking Needs,Ask Foods,85.0,85.0,"Cooking Chocolate, Cocoa",4.0,Chocolate compounds for making chocolate & Cakes.
9128,Bread - Milk,"Bakery, Cakes & Dairy",Breads & Buns,English Oven,40.0,40.0,"Milk, White & Sandwich",NaN,Perfectly baked & fresh
5002,Milk Chocolate,Gourmet & World Food,Chocolates & Biscuits,Hershey's Kisses,650.0,650.0,International Chocolates,4.0,"Make birthday parties, wedding showers, and ot..."


In [223]:

query = 'all-purpose flour'
score,row = similarity_search(query,3)
print(score[0])
print(row[0])

[0.45204583 0.46894988 0.5427946 ]
[9048 8852 4911]


In [224]:
df.loc[row[0]]

,product,category,sub_category,brand,sale_price,market_price,type,rating,description
9048,Unbleached All Purpose Flour,Gourmet & World Food,Cooking & Baking Needs,American Pancake Co.,166.5,185.0,Flours & Pre-Mixes,4.6,At the American Pancake Co. we are all about m...
8852,Wheat Flour,"Foodgrains, Oil & Masala","Atta, Flours & Sooji",Super Saver,686.0,800.0,Atta Whole Wheat,NaN,Atta is made from the choicest grains - heavy ...
4911,Corn Flour,Gourmet & World Food,Cooking & Baking Needs,Puramate,22.5,25.0,Flours & Pre-Mixes,4.3,Puramate Corn flour blends easily with liquids...


In [9]:
groq_api_key=os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.1-70b-versatile")

llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000024F2F79AB60>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000024F2F7998D0>, model_name='llama-3.1-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

# chat bot


In [10]:
#please dont provide any measurement, u messed up flour and all purpose flour be carefull
qa_prompt = ChatPromptTemplate.from_messages([
    ("system", '''
     "u are a helpful ai assitant, provide "
     "name": "Recipe Name",
            "ingredients": ["Ingredient 1", "Ingredient 2"] ,
            "items" :["Ingredient 1", "Ingredient 2"] provide without quantity,dont mess up all-purpose flour with flour)
            "instructions": "Step-by-step instructions.",
            "prep_time": as suggested
     instructions and ingredients to cook a recipe strictly in json format '''),
    ("human", "{input}")
])
# dont trasnform any name of ingredient in item list u messed up flour and all purpose flour be carefull

chain = qa_prompt | llm 
result1 = chain.invoke({"input":"how to bake cake"})

response1 = result1.content
response1

'{\n    "name": "Classic Vanilla Cake",\n    "ingredients": [\n        "all-purpose flour",\n        "baking powder",\n        "baking soda",\n        "salt",\n        "granulated sugar",\n        "unsalted butter",\n        "large eggs",\n        "vanilla extract",\n        "milk"\n    ],\n    "items": [\n        "9-inch round cake pans",\n        "non-stick cooking spray",\n        "mixing bowls",\n        "measuring cups",\n        "electric mixer",\n        "whisk",\n        "rubber spatula"\n    ],\n    "instructions": "Here\'s a step-by-step guide to baking a classic vanilla cake:\\n\\n1. Preheat the oven to 350°F (175°C) and prepare two 9-inch round cake pans with non-stick cooking spray.\\n\\n2. In a medium-sized bowl, whisk together 2 1/4 cups of all-purpose flour, 2 teaspoons of baking powder, 1 teaspoon of baking soda, and 1 teaspoon of salt. Set aside.\\n\\n3. In a large mixing bowl, use an electric mixer to cream together 1 cup of unsalted butter and 1 3/4 cups of granulat

In [182]:
# rag output is bullshit

# system_prompt = ('''
#     "you are a assistant for cooking task."
#     "Use the following pieces of retrieved context to provide ingredients "
#     "also recommend the cooking intrutions"
#     "recommend items from the context "
#     "instructions and ingredients to cook a recipe strictly in json format "
#             "name": "Recipe Name",
#             "ingredients": ["Ingredient 1", "Ingredient 2"] ,
#             "items" :["Ingredient 1", "Ingredient 2"] (only without measurement, u messed up flour and all purpose flour be carefull),
#             "instructions": "Step-by-step instructions.",
#             "prep_time": "as suggested"
#     "\n\n"
#     "{context}" in the provided context  are items of a supermarket and we are creating a item recommender system 
#                  '''
# )


# prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system",system_prompt),
#         ("human","{input}") # during invke input will be a key and the quesiton to the llm will be value
#     ]
# )
# retrievar.search_kwargs = {"k": 10}
# question_answer_chain = create_stuff_documents_chain(llm,prompt)
# rag_chain = create_retrieval_chain(retrievar,question_answer_chain)

# result2 = rag_chain.invoke({"input":"how to cook pizza"})




In [183]:
# class revie_response(BaseModel):
#         food_item: str = Field(description="Name of the food item")
#         recipe: str = Field(description="")
#         ingredients:list [str] = Field(description="list of only ingredients ")
#         cons: str = Field(description="a quick and short explanation of recipe")

# structured_llm = llm.with_structured_output(revie_response)
# output = structured_llm.invoke(response)
# print(output)

In [11]:
import json
import re
def json_response(response):
        json_match = re.search(r"{.*}", response, re.DOTALL)
        if json_match:
                cleaned_output = json_match.group()
                recipe = json.loads(cleaned_output)
                print("Parsed Recipe:", recipe)
        else:
                print("No JSON found!") 
        return recipe
recipe = json_response(response=response1)

Parsed Recipe: {'name': 'Classic Vanilla Cake', 'ingredients': ['all-purpose flour', 'baking powder', 'baking soda', 'salt', 'granulated sugar', 'unsalted butter', 'large eggs', 'vanilla extract', 'milk'], 'items': ['9-inch round cake pans', 'non-stick cooking spray', 'mixing bowls', 'measuring cups', 'electric mixer', 'whisk', 'rubber spatula'], 'instructions': "Here's a step-by-step guide to baking a classic vanilla cake:\n\n1. Preheat the oven to 350°F (175°C) and prepare two 9-inch round cake pans with non-stick cooking spray.\n\n2. In a medium-sized bowl, whisk together 2 1/4 cups of all-purpose flour, 2 teaspoons of baking powder, 1 teaspoon of baking soda, and 1 teaspoon of salt. Set aside.\n\n3. In a large mixing bowl, use an electric mixer to cream together 1 cup of unsalted butter and 1 3/4 cups of granulated sugar until light and fluffy.\n\n4. Beat in 3 large eggs one at a time, allowing each egg to fully incorporate before adding the next.\n\n5. Beat in 2 teaspoons of vanil

In [12]:
recipe_name = recipe.get('name')
recipe_name

'Classic Vanilla Cake'

In [13]:
instructions = recipe.get('instructions')
instructions

"Here's a step-by-step guide to baking a classic vanilla cake:\n\n1. Preheat the oven to 350°F (175°C) and prepare two 9-inch round cake pans with non-stick cooking spray.\n\n2. In a medium-sized bowl, whisk together 2 1/4 cups of all-purpose flour, 2 teaspoons of baking powder, 1 teaspoon of baking soda, and 1 teaspoon of salt. Set aside.\n\n3. In a large mixing bowl, use an electric mixer to cream together 1 cup of unsalted butter and 1 3/4 cups of granulated sugar until light and fluffy.\n\n4. Beat in 3 large eggs one at a time, allowing each egg to fully incorporate before adding the next.\n\n5. Beat in 2 teaspoons of vanilla extract.\n\n6. With the mixer on low speed, gradually add the flour mixture to the wet ingredients in three parts, alternating with 1 cup of milk, beginning and ending with the flour mixture. Beat just until combined.\n\n7. Divide the batter evenly between the prepared pans and smooth the tops.\n\n8. Bake for 30-35 minutes, or until a toothpick inserted into t

In [14]:
ingredients = recipe.get('ingredients')
ingredients

['all-purpose flour',
 'baking powder',
 'baking soda',
 'salt',
 'granulated sugar',
 'unsalted butter',
 'large eggs',
 'vanilla extract',
 'milk']

['All-purpose flour',
 'Granulated sugar',
 'Unsalted butter, softened',
 'Large eggs, at room temperature',
 'Whole milk, at room temperature',
 'Pure vanilla extract']

In [15]:
items = recipe.get('items')
items

['9-inch round cake pans',
 'non-stick cooking spray',
 'mixing bowls',
 'measuring cups',
 'electric mixer',
 'whisk',
 'rubber spatula']

In [19]:
df_copy = filtered_df.copy()

In [ ]:
corpus.index('unbleached all purpose flour')

In [ ]:
def item_search(items):
    df_list = []
    for item in items:
        a=db_load.similarity_search_with_relevance_scores(item,k=1)
        page_content = a[0][0].page_content
        score = a[0][1]
        df_list.append(page_content)
        
item_search(ingredients)

['unbleached all purpose flour']
['unbleached all purpose flour', 'baking powder']
['unbleached all purpose flour', 'baking powder', 'baking soda']
['unbleached all purpose flour', 'baking powder', 'baking soda', 'sea salt']
['unbleached all purpose flour', 'baking powder', 'baking soda', 'sea salt', 'sugar']
['unbleached all purpose flour', 'baking powder', 'baking soda', 'sea salt', 'sugar', 'unsalted butter']
['unbleached all purpose flour', 'baking powder', 'baking soda', 'sea salt', 'sugar', 'unsalted butter', 'eggs']
['unbleached all purpose flour', 'baking powder', 'baking soda', 'sea salt', 'sugar', 'unsalted butter', 'eggs', 'vanilla milkshake with real vanilla extracts']
['unbleached all purpose flour', 'baking powder', 'baking soda', 'sea salt', 'sugar', 'unsalted butter', 'eggs', 'vanilla milkshake with real vanilla extracts', 'chocolate   milk']


In [33]:
df_copy[df_copy['product'].isin(df_list)]

,product,category,sub_category,brand,sale_price,market_price,type,rating,description


In [20]:
def item_search(items):
    df_list = []
    for item in items:
        a=db_load.similarity_search_with_relevance_scores(item,k=1)
        page_content = a[0][0].page_content
        score = a[0][1]
        if score >0.65:
            b = df_copy.index[df_copy[page_content]]
            df_list.append(b)
    return df_list
df_list = item_search(ingredients)

KeyError: 'unbleached all purpose flour'

In [ ]:
df_copy.index[df_copy['']]

In [46]:
df_list

[9048, 5371, 8979, 3764, 4604, 10648, 984, 460]

In [1]:
df_list = [9048, 5371, 8979, 3764, 4604, 10648, 984, 460]

In [47]:
df_copy.loc[df_list]

,product,category,sub_category,brand,sale_price,market_price,type,rating,description
9048,Unbleached All Purpose Flour,Gourmet & World Food,Cooking & Baking Needs,American Pancake Co.,166.50,185.0,Flours & Pre-Mixes,4.6,At the American Pancake Co. we are all about m...
5371,Baking Powder,Gourmet & World Food,Cooking & Baking Needs,Puramate,27.00,30.0,"Baking, Cake Decorations",4.3,Baking Powder is a leavening agent & it reacts...
8979,Baking Soda,Gourmet & World Food,Cooking & Baking Needs,Purix,90.25,95.0,"Baking, Cake Decorations",NaN,Baking Soda a leavening agent & it reacts with...
3764,Sea Salt,"Foodgrains, Oil & Masala","Salt, Sugar & Jaggery",NaturoBell,180.00,180.0,Salts,4.2,Sea Salt Is The Natural Salt That Is Produced ...
4604,Sugar,"Foodgrains, Oil & Masala","Salt, Sugar & Jaggery",bb Popular,202.95,275.0,Sugar & Jaggery,4.0,Sugar is a generalized name for sweet. Sugar i...
10648,Unsalted Butter,"Bakery, Cakes & Dairy",Dairy,Amul,48.00,49.0,Butter & Margarine,4.2,"Butter in its purest form, made of cream with ..."
984,Eggs,"Eggs, Meat & Fish",Eggs,Ayur Plus,75.00,75.0,Farm Eggs,NaN,Ayur Egg+ hens are fed with highly nutritious ...
460,Chocolate - Milk,Gourmet & World Food,Cooking & Baking Needs,Ask Foods,85.00,85.0,"Cooking Chocolate, Cocoa",4.0,Chocolate compounds for making chocolate & Cakes.
